<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%202/images/IDSNlogo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>

# Simple Linear Regression

Estimated time needed: **15** minutes

## Objectives

After completing this lab you will be able to:

*   Use scikit-learn to implement simple Linear Regression
*   Create a model, train it, test it and use the model


### Importing Needed packages


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import pylab as pl
import numpy as np
%matplotlib inline

### Downloading Data

To download the data, we will use !wget to download it from IBM Object Storage.


In [ ]:
!wget -O FuelConsumption.csv https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%202/data/FuelConsumptionCo2.csv

**Did you know?** When it comes to Machine Learning, you will likely be working with large datasets. As a business, where can you host your data? IBM is offering a unique opportunity for businesses, with 10 Tb of IBM Cloud Object Storage: [Sign up now for free](http://cocl.us/ML0101EN-IBM-Offer-CC)


## Understanding the Data

### `FuelConsumption.csv`:

We have downloaded a fuel consumption dataset, **`FuelConsumption.csv`**, which contains model-specific fuel consumption ratings and estimated carbon dioxide emissions for new light-duty vehicles for retail sale in Canada. [Dataset source](http://open.canada.ca/data/en/dataset/98f1a129-f628-4ce4-b24d-6f16bf24dd64?utm_medium=Exinfluencer\&utm_source=Exinfluencer\&utm_content=000026UJ\&utm_term=10006555\&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML0101ENSkillsNetwork20718538-2021-01-01)

*   **MODELYEAR** e.g. 2014
*   **MAKE** e.g. Acura
*   **MODEL** e.g. ILX
*   **VEHICLE CLASS** e.g. SUV
*   **ENGINE SIZE** e.g. 4.7
*   **CYLINDERS** e.g 6
*   **TRANSMISSION** e.g. A6
*   **FUEL CONSUMPTION in CITY(L/100 km)** e.g. 9.9
*   **FUEL CONSUMPTION in HWY (L/100 km)** e.g. 8.9
*   **FUEL CONSUMPTION COMB (L/100 km)** e.g. 9.2
*   **CO2 EMISSIONS (g/km)** e.g. 182   --> low --> 0


## Reading the data in


In [ ]:
df = pd.read_csv("FuelConsumption.csv")

# take a look at the dataset
df.head()



In [ ]:
df['MAKE'].tail()

### Data Exploration

Let's first have a descriptive exploration on our data.


In [ ]:
# summarize the data
df.describe()

Let's select some features to explore more.


In [ ]:
cdf = df[['ENGINESIZE','CYLINDERS','FUELCONSUMPTION_COMB','CO2EMISSIONS']]
cdf.head()

We can plot each of these features:


In [ ]:
viz = cdf[['CYLINDERS','ENGINESIZE','CO2EMISSIONS','FUELCONSUMPTION_COMB']]
viz.hist()
plt.show()

In [ ]:
fuel = cdf[['FUELCONSUMPTION_COMB' , 'CYLINDERS']]
count = 0
for i in list(fuel['CYLINDERS']):
    if i == 12: 
        count +=1
print(count)

In [ ]:
#classic way:

cdf3 = cdf.loc[df['CYLINDERS'] == 3]
cdf4 = cdf.loc[df['CYLINDERS'] == 4]
cdf5 = cdf.loc[df['CYLINDERS'] == 5]
cdf6 = cdf.loc[df['CYLINDERS'] == 6]
cdf7 = cdf.loc[df['CYLINDERS'] == 7]
cdf8 = cdf.loc[df['CYLINDERS'] == 8]
cdf10 = cdf.loc[df['CYLINDERS'] == 10]
cdf12 = cdf.loc[df['CYLINDERS'] == 12]


### OH I really found a nice way to name rows in a loop:

In [ ]:
for k in cdf['CYLINDERS']:
    exec(f'ddf{k} = cdf.loc[df[\'CYLINDERS\'] == k] ')
print(ddf12)

# Create a dictionary for every Cylinder:

In [104]:
d = {}
for x in cdf['CYLINDERS']:
    d["Cylinder{0}".format(x)] = cdf.loc[df['CYLINDERS'] == x]
print(d['Cylinder4'])
print(d['Cylinder12'])


      ENGINESIZE  CYLINDERS  FUELCONSUMPTION_COMB  CO2EMISSIONS
0            2.0          4                   8.5           196
1            2.4          4                   9.6           221
2            1.5          4                   5.9           136
9            2.4          4                   9.2           212
10           2.4          4                   9.8           225
...          ...        ...                   ...           ...
1048         2.0          4                   7.1           192
1049         2.0          4                   6.8           184
1051         2.0          4                  10.7           246
1052         2.0          4                  11.6           267
1053         2.0          4                  10.7           246

[420 rows x 4 columns]
     ENGINESIZE  CYLINDERS  FUELCONSUMPTION_COMB  CO2EMISSIONS
12          5.9         12                  15.6           359
13          5.9         12                  15.6           359
18          5.9    

Now, let's plot each of these features against the Emission, to see how linear their relationship is:


In [ ]:

plt.scatter( cdf.FUELCONSUMPTION_COMB, cdf.CO2EMISSIONS,  color='blue')

plt.xlabel("FUELCONSUMPTION_COMB")
plt.ylabel("Emission")
plt.show()

In [ ]:
plt.scatter( cdf3.FUELCONSUMPTION_COMB, cdf3.CO2EMISSIONS,  color='blue' , label = '3 Cylinders')
plt.scatter( cdf4.FUELCONSUMPTION_COMB, cdf4.CO2EMISSIONS,  color='red' , label = '4 Cylinders')
plt.scatter( cdf5.FUELCONSUMPTION_COMB, cdf5.CO2EMISSIONS,  color='yellow' , label = '5 Cylinders')
plt.scatter( cdf6.FUELCONSUMPTION_COMB, cdf6.CO2EMISSIONS,  color='green' , label = '6 Cylinders')
plt.scatter( cdf7.FUELCONSUMPTION_COMB, cdf7.CO2EMISSIONS,  color='black' , label = '7 Cylinders')
plt.scatter( cdf8.FUELCONSUMPTION_COMB, cdf8.CO2EMISSIONS,  color='gray' , label = '8 Cylinders')
plt.scatter( cdf10.FUELCONSUMPTION_COMB, cdf10.CO2EMISSIONS,  color='brown' , label = '10 Cylinders')
plt.scatter( cdf12.FUELCONSUMPTION_COMB, cdf12.CO2EMISSIONS,  color='pink' , label = '12 Cylinders')

plt.xlabel("FUELCONSUMPTION_COMB")
plt.ylabel("Emission")
plt.legend(loc='lower right')
plt.show()



for i in cdf['CYLINDERS']:
    print(i)

In [ ]:
plt.scatter(cdf.ENGINESIZE, cdf.CO2EMISSIONS,  color='blue')
plt.xlabel("Engine size")
plt.ylabel("Emission")
plt.show()

## Practice

Plot **CYLINDER** vs the Emission, to see how linear is their relationship is:


In [ ]:
# write your code here




<details><summary>Click here for the solution</summary>

```python
plt.scatter(cdf.CYLINDERS, cdf.CO2EMISSIONS, color='blue')
plt.xlabel("Cylinders")
plt.ylabel("Emission")
plt.show()

```

</details>


#### Creating train and test dataset

Train/Test Split involves splitting the dataset into training and testing sets that are mutually exclusive. After which, you train with the training set and test with the testing set.
This will provide a more accurate evaluation on out-of-sample accuracy because the testing dataset is not part of the dataset that have been used to train the model. Therefore, it gives us a better understanding of how well our model generalizes on new data.

This means that we know the outcome of each data point in the testing dataset, making it great to test with! Since this data has not been used to train the model, the model has no knowledge of the outcome of these data points. So, in essence, it is truly an out-of-sample testing.

Let's split our dataset into train and test sets. 80% of the entire dataset will be used for training and 20% for testing. We create a mask to select random rows using **np.random.rand()** function:


In [ ]:
msk = np.random.rand(len(cdf)) < 0.8
train = cdf[msk]
test = cdf[~msk]
print(msk)
print(len(cdf))
# print(train)
# print(test)

### Simple Regression Model

Linear Regression fits a linear model with coefficients B = (B1, ..., Bn) to minimize the 'residual sum of squares' between the actual value y in the dataset, and the predicted value that using linear approximation.


#### Train data distribution


In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)
ax1.scatter(train.ENGINESIZE, train.CO2EMISSIONS,  color='blue' , label = 'train')
ax1.scatter(test.ENGINESIZE, test.CO2EMISSIONS,  color='red' , label = 'test')

plt.xlabel("Engine size")
plt.ylabel("Emission")
plt.show()

In [ ]:
# fig = plt.figure()
# ax1 = fig.add_subplot(111)
plt.scatter(train.ENGINESIZE, train.CO2EMISSIONS,  color='blue' , label = 'train')
plt.scatter(test.ENGINESIZE, test.CO2EMISSIONS,  color='red' , label = 'test')

plt.xlabel("Engine size")
plt.ylabel("Emission")
plt.legend(loc='lower right')

plt.show()


#### Modeling

Using sklearn package to model data.


In [ ]:
from sklearn import linear_model
regr = linear_model.LinearRegression()
train_x = np.asanyarray(train[['ENGINESIZE']])
train_y = np.asanyarray(train[['CO2EMISSIONS']])
regr.fit (train_x, train_y)
# The coefficients
print ('Coefficients: ', regr.coef_)
print ('Intercept: ',regr.intercept_)

As mentioned before, **Coefficient** and **Intercept** in the simple linear regression, are the parameters of the fit line.
Given that it is a simple linear regression, with only 2 parameters, and knowing that the parameters are the intercept and slope of the line, sklearn can estimate them directly from our data.
Notice that all of the data must be available to traverse and calculate the parameters.


#### Plot outputs


We can plot the fit line over the data:


In [ ]:
plt.scatter(train.ENGINESIZE, train.CO2EMISSIONS,  color='blue')
plt.plot(train_x, regr.coef_[0][0]*train_x + regr.intercept_[0], '-r')
plt.xlabel("Engine size")
plt.ylabel("Emission")

#### Evaluation

We compare the actual values and predicted values to calculate the accuracy of a regression model. Evaluation metrics provide a key role in the development of a model, as it provides insight to areas that require improvement.

There are different model evaluation metrics, lets use MSE here to calculate the accuracy of our model based on the test set:

*   Mean Absolute Error: It is the mean of the absolute value of the errors. This is the easiest of the metrics to understand since it’s just average error.

*   Mean Squared Error (MSE): Mean Squared Error (MSE) is the mean of the squared error. It’s more popular than Mean Absolute Error because the focus is geared more towards large errors. This is due to the squared term exponentially increasing larger errors in comparison to smaller ones.

*   Root Mean Squared Error (RMSE).

*   R-squared is not an error, but rather a popular metric to measure the performance of your regression model. It represents how close the data points are to the fitted regression line. The higher the R-squared value, the better the model fits your data. The best possible score is 1.0 and it can be negative (because the model can be arbitrarily worse).


In [ ]:
from sklearn.metrics import r2_score

test_x = np.asanyarray(test[['ENGINESIZE']])
test_y = np.asanyarray(test[['CO2EMISSIONS']])
test_y_ = regr.predict(test_x)

print("Mean absolute error: %.2f" % np.mean(np.absolute(test_y_ - test_y)))
print("Residual sum of squares (MSE): %.2f" % np.mean((test_y_ - test_y) ** 2))
print("R2-score: %.2f" % r2_score(test_y , test_y_) )

<h2>Want to learn more?</h2>

IBM SPSS Modeler is a comprehensive analytics platform that has many machine learning algorithms. It has been designed to bring predictive intelligence to decisions made by individuals, by groups, by systems – by your enterprise as a whole. A free trial is available through this course, available here: <a href="https://www.ibm.com/analytics/spss-statistics-software?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML0101ENSkillsNetwork20718538-2021-01-01">SPSS Modeler</a>

Also, you can use Watson Studio to run these notebooks faster with bigger datasets. Watson Studio is IBM's leading cloud solution for data scientists, built by data scientists. With Jupyter notebooks, RStudio, Apache Spark and popular libraries pre-packaged in the cloud, Watson Studio enables data scientists to collaborate on their projects without having to install anything. Join the fast-growing community of Watson Studio users today with a free account at <a href="https://www.ibm.com/cloud/watson-studio?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML0101ENSkillsNetwork20718538-2021-01-01">Watson Studio</a>


### Thank you for completing this lab!

## Author

Saeed Aghabozorgi

### Other Contributors

<a href="https://www.linkedin.com/in/joseph-s-50398b136/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML0101ENSkillsNetwork20718538-2021-01-01" target="_blank">Joseph Santarcangelo</a>

## Change Log

|  Date (YYYY-MM-DD) |  Version | Changed By  |  Change Description |
|---|---|---|---|
| 2020-11-03 | 2.1  | Lakshmi Holla  |  Changed URL of the csv |
| 2020-08-27  | 2.0  | Lavanya  |  Moved lab to course repo in GitLab |
|   |   |   |   |
|   |   |   |   |

## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>
